In [67]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *
from MPRA_predict.metrics import *

from collections import Counter
import re

In [15]:
df = pd.read_csv('../data/Sei/Sei_tracks_info.csv')

df = df[df['AUROC'] > 0.95]

df_pivot = df.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)
df_pivot = df_pivot.map(lambda x: x if isinstance(x, list) else [])
df_pivot

assay                      ADNP AEBP2 AFF1 AFF4 AGO1 AGO2 AHR AHRR AP-2alpha AP-2gamma APOBEC3B  AR ARID1A ARID2 ARID3A ARID5B ARNT ARNT2 ARNTL ARRB1 ASCL1 ASCL2 ASH1L ASH2L ASXL1 ATAC-seq ATF1 ATF2 ATF3 ATF4 ATF5 ATF7IP ATOH1 AUTS2 BACH1 BACH2 BAF155 BAF170 BAHD1 BANF1 BANP BARX1 BARX2 BATF BCAT1 BCL11A BCL11B BCL3 BCL6 BCLAF1 BCOR BDP1 BHLHE40 BIRA BMI1 BPTF BRCA1 BRD1 BRD2 BRD3 BRD4 BRD7 BRD9 BRDU BRPF3 BTAF1 Bach1 Brg1 C11orf30 C17orf49 C17orf96 CAMTA2 CASP8AP2 CBFA2T2 CBFB CBX1 CBX2 CBX3 CBX5 CBX6 CBX7 CBX8 CCDC101 CCND2 CCNT2 CD74 CDC5L CDCA2 CDK12 CDK6 CDK7 CDK8 CDK9 CDX2 CEBPA CEBPB CEBPD CEBPG CEBPZ CENPA CENPC CENPT CHAMP1 CHD1 CHD2 CHD4 CHD5 CHD7 CHD8 CHRM2 CIITA CLOCK CNOT3 COPS2 COREST CPSF3L CREB1 CREB3 CREB3L4 CREBBP CREM CRY1 CSDE1 CSNK2A1 CTBP1 CTBP2     CTCF CTCFL CTNNB1 CUX1 CtBP2 DAXX DCP1A DDX20 DDX21 DDX5 DEAF1 DEK DIDO1 DLX1 DLX2 DNMT1 DNMT3A DNMT3B    DNase DNase.all.peaks DNase.fdr0.01.hot DNase.fdr0.01.peaks DNase.hot DPF1 DPF2 DR1 DRAP1 DROSHA DYRK1A E2F1  \
cell_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
1184_Stem_cell               []    []   []   []   []   []  []   []        []        []       []  []     []    []     []     []   []    []    []    []    []    []    []    []    []       []   []   []   []   []   []     []    []    []    []    []     []     []    []    []   []    []    []   []    []     []     []   []   []     []   []   []      []   []   []   []    []   []   []   []   []   []   []   []    []    []    []   []       []       []       []     []       []      []   []   []   []   []   []   []   []   []      []    []    []   []    []    []    []   []   []   []   []   []    []    []    []    []    []    []    []    []     []   []   []   []   []   []   []    []    []    []    []    []     []     []    []    []      []     []   []   []    []      []    []    []       []    []     []   []    []   []    []    []    []   []    []  []    []   []   []    []     []     []       []              []                []                  []        []   []   []  []    []     []     []   []   
133_Astrocyte                []    []   []   []   []   []  []   []        []        []       []  []     []    []     []     []   []    []    []    []    []    []    []    []    []       []   []   []   []   []   []     []    []    []    []    []     []     []    []    []   []    []    []   []    []     []     []   []   []     []   []   []      []   []   []   []    []   []   []   []   []   []   []   []    []    []    []   []       []       []       []     []       []      []   []   []   []   []   []   []   []   []      []    []    []   []    []    []    []   []   []   []   []   []    []    []    []    []    []    []    []    []     []   []   []   []   []   []   []    []    []    []    []    []     []     []    []    []      []     []   []   []    []      []    []    []       []    []     []   []    []   []    []    []    []   []    []  []    []   []   []    []     []     []       []              []                []                  []        []   []   []  []    []     []     []   []   


In [16]:
# # assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

# assays = ['DNase', 'H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'CTCF']

# cell_types = ['K562', 'HepG2']
# cell_types_standard_names = ['K562_Leukemia_Cell', 'HepG2_Hepatocellular_Carcinoma']

In [41]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'CTCF']

cell_types = ['K562', 'HepG2', 'WTC11']
cell_types_standard_names = ['K562_Leukemia_Cell', 'HepG2_Hepatocellular_Carcinoma', 'iPSC']

In [38]:
df_track = df_pivot.loc[cell_types_standard_names][assays]
df_track.index = cell_types
df_track.map(lambda x: len(x))

assay,DNase,H3K4me3,H3K27ac,H3K27me3,H3K4me1,H3K36me3,H3K9me3,CTCF
K562,57,87,23,5,7,3,15,29
HepG2,9,8,8,2,1,2,0,17
WTC11,5,13,6,7,8,2,2,0


In [34]:
######### Gosai_MPRA first compute normalize params
pred_array = load_h5('outputs/Gosai_MPRA_Sei_pred.h5')
print(pred_array.shape)

MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_my_processed_data_len200_norm.csv')
print(MPRA_df.shape)

(760679, 21907)
(760679, 14)


KeyError: 'K562'

In [39]:
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]
        if len(indice) > 0:
            pred = logit(pred_array[:, indice]).mean(1)
            MPRA_df[f'{cell_type}_{assay}'] = pred
        else:
            MPRA_df[f'{cell_type}_{assay}'] = np.nan
MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,A549,HCT116,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF,WTC11_DNase,WTC11_H3K4me3,WTC11_H3K27ac,WTC11_H3K27me3,WTC11_H3K4me1,WTC11_H3K36me3,WTC11_H3K9me3,WTC11_CTCF
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.014,-0.054,0.012,NaN,-0.709,-3.729,-5.499,-5.620,-7.278,-5.233,-8.731,-2.985,-5.719,-4.818,-4.984,-6.853,-3.876,-6.347,-7.907,NaN,-5.720,-4.043,-5.637,-6.632,-6.011,-6.386,-5.937,-2.307,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-1.042,0.183,0.094,NaN,-0.266,-3.687,-5.445,-5.604,-7.250,-5.224,-8.707,-2.924,-5.765,-4.805,-4.937,-6.846,-3.891,-6.413,-7.923,NaN,-5.793,-3.993,-5.552,-6.613,-5.918,-6.369,-5.919,-2.350,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-1.004,0.640,-0.188,NaN,0.619,-5.616,-8.213,-8.650,-8.338,-7.815,-10.701,-6.840,-8.355,-4.805,-6.109,-7.566,-6.314,-5.830,-10.344,NaN,-7.261,-3.756,-7.954,-7.739,-8.292,-7.465,-8.535,-6.258,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.779,0.751,0.274,NaN,0.668,-5.402,-7.903,-8.314,-8.168,-7.640,-10.673,-6.845,-8.255,-4.791,-5.761,-7.334,-6.196,-5.684,-10.297,NaN,-7.263,-3.624,-7.749,-7.418,-8.213,-7.386,-8.689,-6.584,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.457,-0.645,0.061,NaN,-0.168,-8.219,-10.384,-9.864,-9.135,-9.694,-12.692,-8.637,-10.799,-9.580,-9.774,-10.401,-7.371,-9.600,-11.485,NaN,-11.242,-5.827,-10.561,-7.918,-9.802,-7.580,-10.318,-9.419,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,0.708,1.294,1.077,NaN,NaN,-1.869,-4.290,-5.463,-6.614,-4.801,-7.704,-1.766,-5.451,-2.745,-3.408,-6.452,-4.101,-5.933,-7.847,NaN,-5.345,-0.535,-3.068,-4.982,-4.877,-5.397,-5.645,-2.651,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.708,-0.547,-0.429,NaN,NaN,-7.313,-10.187,-9.871,-9.353,-9.003,-11.804,-8.000,-10.233,-8.482,-10.009,-10.216,-7.432,-8.972,-10.877,NaN,-10.184,-6.705,-10.698,-9.759,-9.516,-10.102,-9.528,-7.633,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,-0.146,-0.200,-0.310,NaN,NaN,-6.397,-9.274,-8.510,-9.010,-7.823,-10.773,-7.456,-9.099,-6.260,-8.046,-8.115,-7.010,-6.637,-10.394,NaN,-8.128,-4.809,-9.076,-7.623,-8.475,-7.637,-8.803,-6.455,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,-0.119,-0.090,-0.387,NaN,NaN,-6.192,-9.311,-9.060,-8.356,-7.782,-9.835,-7.284,-9.574,-7.647,-9.164,-9.711,-6.322,-7.972,-9.490,NaN,-9.199,-4.329,-8.813,-7.285,-8.740,-6.692,-8.284,-5.340,NaN


In [42]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['chrom'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(1, 23)) + ['X', 'Y'])])
data_split_dict['designed'] = ~data_split_dict['chrom']

data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test'] = MPRA_df['chr'].isin(['chr2'])



norm_params = pd.DataFrame()
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        
        x = MPRA_df[f'{cell_type}_{assay}']
        condition = data_split_dict['train']
        x_train = x[condition]

        x_mean = np.mean(x_train)
        x_std = np.std(x_train)
        x = (x - x_mean) / x_std

        norm_params.loc['mean', f'{cell_type}_{assay}'] = x_mean
        norm_params.loc['std', f'{cell_type}_{assay}'] = x_std
norm_params

,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF,WTC11_DNase,WTC11_H3K4me3,WTC11_H3K27ac,WTC11_H3K27me3,WTC11_H3K4me1,WTC11_H3K36me3,WTC11_H3K9me3,WTC11_CTCF
mean,-6.448,-9.009,-8.816,-9.088,-8.194,-11.218,-6.377,-8.838,-7.458,-8.465,-9.263,-6.803,-8.587,-10.250,NaN,-8.704,-5.477,-9.252,-8.616,-8.792,-8.811,-8.700,-6.536,NaN
std,1.957,1.702,1.614,0.917,1.562,1.290,1.802,1.989,2.152,1.904,1.530,1.252,1.688,1.102,NaN,2.235,1.746,1.967,1.502,1.260,1.651,1.305,2.211,NaN


In [68]:
####### Agarwal_MPRA

pred_array = load_h5('outputs/Agarwal_MPRA_Sei_pred.h5')
print(pred_array.shape)

(56975, 21907)


In [69]:
MPRA_df = pd.read_csv('../data/Agarwal_MPRA/Agarwal_MPRA_joint_56k.csv')
print(MPRA_df.shape)

(56975, 5)


In [70]:
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]

        if len(indice) > 0:
            for idx in indice:
                pred = logit(pred_array[:, idx])
                # pred = pred_array[:, idx]
                true = MPRA_df[cell_type]
                r, _ = pearson(pred, true)
                print(cell_type, assay, idx, r)

K562 DNase 1490 0.23634473653657687
K562 DNase 2919 0.40183563461205385
K562 DNase 3167 0.4763686683163797
K562 DNase 4632 0.47667583220975873
K562 DNase 4633 0.4606272499084298
K562 DNase 10715 0.4648229103227851
K562 DNase 10716 0.47764334570991696
K562 DNase 10717 0.4661945889310495
K562 DNase 10718 0.39706811401439435
K562 DNase 10719 0.4802493056316183
K562 DNase 10720 0.4408137361556317


K562 DNase 10721 0.4160031370508047
K562 DNase 10722 0.4362234353842961
K562 DNase 10723 0.4538683437762553
K562 DNase 10724 0.43779807097886386
K562 DNase 10725 0.4755018546580194
K562 DNase 10726 0.43351294262982787
K562 DNase 10727 0.4647490313778555
K562 DNase 10728 0.5458300545071509
K562 DNase 10729 0.4231547849437789
K562 DNase 10730 0.39867679055624733
K562 DNase 10731 0.4336591843523095
K562 DNase 10732 0.4516785083269676
K562 DNase 10749 0.4389411587992431
K562 DNase 10817 0.4202258204349552
K562 DNase 10822 0.5574108601842458
K562 DNase 10857 0.48932209432116447
K562 DNase 10954 0.4377537632500525
K562 DNase 10991 0.477389015001339
K562 DNase 11064 0.42889539774016183
K562 DNase 11100 0.4097982925454264
K562 DNase 11164 0.4472542800693366
K562 DNase 11383 0.43766010990178605
K562 DNase 11427 0.48820021282351317
K562 DNase 11492 0.5091435649614247
K562 DNase 11519 0.4314282620136758
K562 DNase 11530 0.46352056294583016
K562 DNase 11554 0.4179541113578401
K562 DNase 11625 0.45

In [71]:
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]
        if len(indice) > 0:
            pred = logit(pred_array[:, indice]).mean(1)
            true = MPRA_df[cell_type]
            MPRA_df[f'{cell_type}_{assay}'] = pred
        else:
            MPRA_df[f'{cell_type}_{assay}'] = np.nan

MPRA_df

,seq,chr,HepG2,K562,WTC11,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF,WTC11_DNase,WTC11_H3K4me3,WTC11_H3K27ac,WTC11_H3K27me3,WTC11_H3K4me1,WTC11_H3K36me3,WTC11_H3K9me3,WTC11_CTCF
0,AGGACCGGATCAACTGTAGGACCAGGCTCTGCCTCCTTCTCTCTCC...,chr1,0.633,0.080,-0.387,-3.445,-5.502,-5.511,-7.763,-5.572,-8.423,-2.810,-6.952,-4.787,-4.764,-6.896,-4.622,-6.765,-7.712,NaN,-6.770,-2.873,-5.731,-5.473,-7.722,-4.708,-5.818,-1.954,NaN
1,AGGACCGGATCAACTTGTACTTGCCTTAGGAGGTCAAACACATTAG...,chr10,-1.157,-1.147,-1.507,-5.282,-7.402,-7.991,-7.237,-6.490,-9.587,-6.234,-7.719,-6.022,-6.713,-8.123,-4.510,-6.486,-8.945,NaN,-7.284,-4.151,-6.689,-7.142,-6.734,-5.932,-7.275,-3.978,NaN
2,AGGACCGGATCAACTTAGCTCAACACAAATCCATCACTGAAAGCAG...,chr10,-0.066,0.186,0.655,-0.028,-6.980,-7.964,-8.695,-5.819,-8.647,-5.215,2.296,-1.032,-5.075,-7.230,-7.291,-6.541,-8.492,NaN,3.947,2.584,-5.658,-7.081,-7.356,-7.296,-7.617,-5.917,NaN
3,AGGACCGGATCAACTGACCCTAAATCAGTATGGGGAAAGGTCCCCA...,chr10,0.846,1.212,-0.637,-2.658,-6.061,-5.540,-7.881,-4.457,-9.559,-5.215,-6.732,-3.127,-4.994,-5.873,-5.681,-5.394,-8.944,NaN,-6.267,-0.513,-5.184,-3.520,-6.712,-4.231,-7.227,-5.867,NaN
4,AGGACCGGATCAACTGTGATCTCAGCTTACTGCAACCTCTGCCTCC...,chr10,-0.784,-0.630,-0.612,-5.119,-10.031,-10.358,-10.663,-8.992,-11.233,-4.821,-2.716,-7.523,-10.271,-11.180,-9.262,-11.038,-10.586,NaN,-1.866,-4.749,-11.401,-12.007,-10.051,-12.048,-8.949,-7.344,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56970,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.192,-1.080,-1.680,-5.319,-6.992,-7.070,-7.979,-6.952,-9.888,-5.629,-8.014,-6.993,-6.509,-8.015,-4.938,-7.691,-9.093,NaN,-8.142,-5.344,-7.584,-8.010,-7.531,-8.080,-7.431,-4.699,NaN
56971,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-0.582,-1.094,-1.637,-4.324,-6.630,-6.814,-8.235,-6.292,-10.353,-6.715,-8.557,-5.411,-5.773,-7.432,-6.196,-5.608,-10.255,NaN,-8.324,-5.476,-8.714,-8.798,-8.542,-8.491,-8.761,-5.891,NaN
56972,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCATGTTTGCTTT...,chrnan,-0.855,-0.554,-1.083,-4.692,-7.021,-6.936,-8.775,-6.277,-10.130,-6.657,-7.469,0.817,-4.199,-4.801,-6.771,-3.563,-10.230,NaN,-4.578,-3.739,-7.393,-8.148,-6.386,-6.646,-8.253,-8.571,NaN
56973,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.231,-0.927,-1.606,-6.376,-8.388,-8.564,-8.488,-7.935,-10.844,-6.585,-8.890,-7.014,-7.302,-8.406,-5.795,-7.270,-10.181,NaN,-8.645,-5.823,-8.742,-8.492,-8.417,-8.501,-8.664,-6.185,NaN


In [72]:
pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        pred = MPRA_df[f'{cell_type}_{assay}']
        true = MPRA_df[cell_type]
        r, _ = pearson(pred, true)
        pearson_df.loc[cell_type, assay] = r

pearson_df

after remove nan, len(x) < 2, pearson = nan
after remove nan, len(x) < 2, pearson = nan


,DNase,H3K4me3,H3K27ac,H3K27me3,H3K4me1,H3K36me3,H3K9me3,CTCF
K562,0.451,0.458,0.411,0.018,0.364,0.333,0.250,0.076
HepG2,0.334,0.339,0.348,-0.011,0.290,0.221,NaN,-0.034
WTC11,0.444,0.392,0.275,0.116,0.203,0.285,0.132,NaN


In [73]:
# assay = 'DNase'
# cell_types_assay = [f'{cell_type}_{assay}' for cell_type in cell_types]

# pearson_df = pd.DataFrame(index=cell_types, columns=cell_types_assay)
# for cell_type in cell_types:
#     for cell_type_pred in cell_types_assay:
#         df = MPRA_df
#         x = df[f'{cell_type}']
#         y = df[f'{cell_type_pred}']
#         r = pearson(x, y)[0]
#         pearson_df.loc[cell_type, cell_type_pred] = r

# print(pearson_df)

In [58]:
MPRA_df['WTC11_CTCF'] = 0

In [59]:
MPRA_df.to_csv('../data/Agarwal_MPRA/Agarwal_MPRA_Sei_feature_0601.csv', index=False)

In [99]:
MPRA_df = pd.read_csv('../data/Agarwal_MPRA/Agarwal_MPRA_Sei_feature_0601.csv')
MPRA_df

,seq,chr,HepG2,K562,WTC11,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF,WTC11_DNase,WTC11_H3K4me3,WTC11_H3K27ac,WTC11_H3K27me3,WTC11_H3K4me1,WTC11_H3K36me3,WTC11_H3K9me3,WTC11_CTCF
0,AGGACCGGATCAACTGTAGGACCAGGCTCTGCCTCCTTCTCTCTCC...,chr1,0.633,0.080,-0.387,-3.445,-5.502,-5.511,-7.763,-5.572,-8.423,-2.810,-6.952,-4.787,-4.764,-6.896,-4.622,-6.765,-7.712,NaN,-6.770,-2.873,-5.731,-5.473,-7.722,-4.708,-5.818,-1.954,0
1,AGGACCGGATCAACTTGTACTTGCCTTAGGAGGTCAAACACATTAG...,chr10,-1.157,-1.147,-1.507,-5.282,-7.402,-7.991,-7.237,-6.490,-9.587,-6.234,-7.719,-6.022,-6.713,-8.123,-4.510,-6.486,-8.945,NaN,-7.284,-4.151,-6.689,-7.142,-6.734,-5.932,-7.275,-3.978,0
2,AGGACCGGATCAACTTAGCTCAACACAAATCCATCACTGAAAGCAG...,chr10,-0.066,0.186,0.655,-0.028,-6.980,-7.964,-8.695,-5.819,-8.647,-5.215,2.296,-1.032,-5.075,-7.230,-7.291,-6.541,-8.492,NaN,3.947,2.584,-5.658,-7.081,-7.356,-7.296,-7.617,-5.917,0
3,AGGACCGGATCAACTGACCCTAAATCAGTATGGGGAAAGGTCCCCA...,chr10,0.846,1.212,-0.637,-2.658,-6.061,-5.540,-7.881,-4.457,-9.559,-5.215,-6.732,-3.127,-4.994,-5.873,-5.681,-5.394,-8.944,NaN,-6.267,-0.513,-5.184,-3.520,-6.712,-4.231,-7.227,-5.867,0
4,AGGACCGGATCAACTGTGATCTCAGCTTACTGCAACCTCTGCCTCC...,chr10,-0.784,-0.630,-0.612,-5.119,-10.031,-10.358,-10.663,-8.992,-11.233,-4.821,-2.716,-7.523,-10.271,-11.180,-9.262,-11.038,-10.586,NaN,-1.866,-4.749,-11.401,-12.007,-10.051,-12.048,-8.949,-7.344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56970,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.192,-1.080,-1.680,-5.319,-6.992,-7.070,-7.979,-6.952,-9.888,-5.629,-8.014,-6.993,-6.509,-8.015,-4.938,-7.691,-9.093,NaN,-8.142,-5.344,-7.584,-8.010,-7.531,-8.080,-7.431,-4.699,0
56971,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-0.582,-1.094,-1.637,-4.324,-6.630,-6.814,-8.235,-6.292,-10.353,-6.715,-8.557,-5.411,-5.773,-7.432,-6.196,-5.608,-10.255,NaN,-8.324,-5.476,-8.714,-8.798,-8.542,-8.491,-8.761,-5.891,0
56972,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCATGTTTGCTTT...,chrnan,-0.855,-0.554,-1.083,-4.692,-7.021,-6.936,-8.775,-6.277,-10.130,-6.657,-7.469,0.817,-4.199,-4.801,-6.771,-3.563,-10.230,NaN,-4.578,-3.739,-7.393,-8.148,-6.386,-6.646,-8.253,-8.571,0
56973,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.231,-0.927,-1.606,-6.376,-8.388,-8.564,-8.488,-7.935,-10.844,-6.585,-8.890,-7.014,-7.302,-8.406,-5.795,-7.270,-10.181,NaN,-8.645,-5.823,-8.742,-8.492,-8.417,-8.501,-8.664,-6.185,0


In [100]:
data_split_dict = {}
data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['chrom'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(1, 23)) + ['X', 'Y'])])
data_split_dict['designed'] = ~data_split_dict['chrom']

data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test'] = MPRA_df['chr'].isin(['chr2'])

In [101]:
cols = [f'{cell_type}_{assay}' for cell_type in cell_types for assay in assays]

x = MPRA_df[cols]

cond = data_split_dict['train']
x_train = MPRA_df.loc[cond, cols].values

x_mean = x_train.mean(axis=0)
x_std = x_train.std(axis=0)
x = (x - x_mean) / x_std

MPRA_df[cols] = x
MPRA_df

,seq,chr,HepG2,K562,WTC11,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF,WTC11_DNase,WTC11_H3K4me3,WTC11_H3K27ac,WTC11_H3K27me3,WTC11_H3K4me1,WTC11_H3K36me3,WTC11_H3K9me3,WTC11_CTCF
0,AGGACCGGATCAACTGTAGGACCAGGCTCTGCCTCCTTCTCTCTCC...,chr1,0.633,0.080,-0.387,-0.595,0.322,0.385,0.157,-0.135,0.580,1.024,-0.687,-0.750,0.193,-0.061,0.843,-0.727,0.996,NaN,-0.759,-0.469,0.455,0.796,-0.543,1.477,1.361,1.817,NaN
1,AGGACCGGATCAACTTGTACTTGCCTTAGGAGGTCAAACACATTAG...,chr10,-1.157,-1.147,-1.507,-1.422,-0.890,-1.064,0.840,-0.835,-0.556,-1.052,-0.881,-1.287,-0.984,-0.784,0.934,-0.525,-0.300,NaN,-0.875,-0.954,-0.107,-0.323,0.317,0.490,-0.010,0.547,NaN
2,AGGACCGGATCAACTTAGCTCAACACAAATCCATCACTGAAAGCAG...,chr10,-0.066,0.186,0.655,0.944,-0.620,-1.049,-1.053,-0.323,0.362,-0.435,1.651,0.886,0.005,-0.258,-1.309,-0.565,0.177,NaN,1.663,1.599,0.497,-0.282,-0.225,-0.610,-0.331,-0.670,NaN
3,AGGACCGGATCAACTGACCCTAAATCAGTATGGGGAAAGGTCCCCA...,chr10,0.846,1.212,-0.637,-0.241,-0.035,0.368,0.004,0.715,-0.528,-0.435,-0.632,-0.026,0.054,0.543,-0.011,0.263,-0.299,NaN,-0.645,0.425,0.775,2.106,0.336,1.861,0.035,-0.639,NaN
4,AGGACCGGATCAACTGTGATCTCAGCTTACTGCAACCTCTGCCTCC...,chr10,-0.784,-0.630,-0.612,-1.349,-2.566,-2.448,-3.607,-2.743,-2.162,-0.196,0.384,-1.941,-3.134,-2.587,-2.899,-3.812,-2.024,NaN,0.349,-1.180,-2.871,-3.586,-2.571,-4.442,-1.583,-1.566,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56970,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.192,-1.080,-1.680,-1.439,-0.628,-0.526,-0.123,-1.188,-0.849,-0.686,-0.956,-1.710,-0.861,-0.721,0.588,-1.395,-0.455,NaN,-1.069,-1.406,-0.632,-0.906,-0.377,-1.242,-0.156,0.094,NaN
56971,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-0.582,-1.094,-1.637,-0.991,-0.398,-0.377,-0.456,-0.684,-1.303,-1.345,-1.093,-1.021,-0.416,-0.376,-0.426,0.109,-1.677,NaN,-1.110,-1.456,-1.295,-1.434,-1.257,-1.573,-1.407,-0.654,NaN
56972,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCATGTTTGCTTT...,chrnan,-0.855,-0.554,-1.083,-1.157,-0.647,-0.448,-1.157,-0.672,-1.085,-1.309,-0.818,1.691,0.535,1.175,-0.889,1.586,-1.650,NaN,-0.264,-0.798,-0.520,-0.998,0.619,-0.086,-0.929,-2.336,NaN
56973,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.231,-0.927,-1.606,-1.915,-1.518,-1.399,-0.784,-1.937,-1.782,-1.266,-1.177,-1.719,-1.340,-0.951,-0.103,-1.091,-1.599,NaN,-1.183,-1.587,-1.312,-1.228,-1.148,-1.582,-1.315,-0.838,NaN


In [102]:
MPRA_df['WTC11_CTCF'] = 0

In [103]:
pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        pred = MPRA_df[f'{cell_type}_{assay}']
        true = MPRA_df[cell_type]
        r, _ = pearson(pred, true)
        pearson_df.loc[cell_type, assay] = r

pearson_df

after remove nan, len(x) < 2, pearson = nan


/home/hxcai/cell_type_specific_CRE/MPRA_predict/predict_epi_features/../MPRA_predict/utils/data_utils.py:118: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = pearsonr(x, y)


,DNase,H3K4me3,H3K27ac,H3K27me3,H3K4me1,H3K36me3,H3K9me3,CTCF
K562,0.451,0.458,0.411,0.018,0.364,0.333,0.250,0.076
HepG2,0.334,0.339,0.348,-0.011,0.290,0.221,NaN,-0.034
WTC11,0.444,0.392,0.275,0.116,0.203,0.285,0.132,NaN


In [104]:
MPRA_df.to_csv('../data/Agarwal_MPRA/Agarwal_MPRA_Sei_feature_norm_0601_2.csv', index=False)